In [97]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
import time

import os
import shutil
import collections
import math
import time
import torchvision
import torch
import torch.nn as nn
import torch.utils.data

import datetime
# !pip install tqdm
from tqdm import tqdm


In [98]:
import torch
from torch import nn

# VGG模型定义
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

def vgg(conv_arch):
    conv_blks = []
    in_channels = 3  # CIFAR-10是3通道彩色图像
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    # 构建卷积层
    net = nn.Sequential(*conv_blks)

    # 计算展平后的尺寸
    with torch.no_grad():
        # 假设输入尺寸为(1, 3, 32, 32)
        sample_input = torch.randn(1, 3, 32, 32)
        sample_output = net(sample_input)
        flattened_size = sample_output.shape[1] * sample_output.shape[2] * sample_output.shape[3]

    # 构建完整的VGG网络
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(flattened_size, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))



# ratio = 4
conv_arch = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))
net = vgg(conv_arch)


In [99]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim
import time
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split


from d2l import torch as d2l  # 确保已经安装了 d2l 包


if torch.cuda.is_available():
    devices = [torch.device('cuda')]
    print(f"Training on {torch.cuda.get_device_name(0)}")
else:
    devices = [torch.device('cpu')]
    print("Training on CPU")


# 数据预处理
transform_train = transforms.Compose([
    transforms.Resize(40),
    transforms.RandomResizedCrop(32, scale=(0.64, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])
batch_size = 128

# 加载 CIFAR-10 数据集
test_ds = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

# 加载 CIFAR-10 训练集
full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

# 分割训练集和验证集
valid_ratio = 0.1  # 比如，使用 10% 的数据作为验证集
num_train = len(full_train_ds)
num_valid = int(num_train * valid_ratio)
train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])

# 创建 DataLoader
train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)


# 定义数据加载器
train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
valid_ratio = 0.1
# 其余定义模型和训练的代码保持不变...
valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)
def get_net():
    num_classes = 10
    net = d2l.resnet18(num_classes, 3)
    return net

loss = nn.CrossEntropyLoss(reduction="none")

def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
          lr_decay):
    trainer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9,
                              weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss', 'train acc']
    if valid_iter is not None:
        legend.append('valid acc')
    # animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
    #                         legend=legend)
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    train_loss_list, train_acc_list, valid_acc_list = [], [], []
    total_time = 0
    for epoch in range(num_epochs):
    # for epoch in tqdm(range(num_epochs), desc='Training Progress'):        
        net.train()
        metric = d2l.Accumulator(3)
        start = time.time()
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = d2l.train_batch_ch13(net, features, labels.to(devices[0]), loss, trainer, devices)
            metric.add(l, acc, labels.shape[0])
            timer.stop()
            # if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                # animator.add(epoch + (i + 1) / num_batches,
                #              (metric[0] / metric[2], metric[1] / metric[2],
                #               None))
        train_loss, train_acc = metric[0] / metric[2], metric[1] / metric[2]                          
        epoch_time = time.time() - start
        total_time += epoch_time
        # print(f'Epoch {epoch + 1}, Loss {metric[0] / metric[2]:.3f}, Train acc {metric[1] / metric[2]:.3f}, Time {timer.sum():.1f} sec')
        if valid_iter is not None:
            valid_acc = d2l.evaluate_accuracy_gpu(net, valid_iter)
            # animator.add(epoch + 1, (None, None, valid_acc))
        print(f"{epoch+1}/{num_epochs} - {epoch_time:.0f}s - loss: {train_loss:.4f} - accuracy: {train_acc:.4f} - val_accuracy: {valid_acc:.4f} - {epoch_time:.0f}s/epoch - {timer.avg():.0f}ms/step")
        scheduler.step()
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)
    measures = (f'train loss {metric[0] / metric[2]:.3f}',f'train acc {metric[1] / metric[2]:.3f}')
    # if valid_iter is not None:
    #     measures += f', valid acc {valid_acc:.3f}'
    print(f'Time {total_time:.1f} sec')
    # print(measures + f'\n{metric[2] * num_epochs / timer.sum():.1f}'
    #       f' examples/sec on {str(devices)}')
    # 保存训练图表
    # save_dir = 'training_plots'
    # os.makedirs(save_dir, exist_ok=True)
    # plot_filename = f"{save_dir}/train_plot_{noise_type}_{noise_level}.png"
    # plt.savefig(plot_filename)
    # plt.close()

    # 绘制训练图表
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_loss_list, label='train_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_acc_list, label='train_acc')
    plt.plot(range(1, num_epochs + 1), valid_acc_list, label='valid_acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # 保存训练图表
    current_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    save_dir = f'training_plots'
    os.makedirs(save_dir, exist_ok=True)
    plot_filename = f"{save_dir}/{current_time}_train_plot_{noise_type}_{noise_level}.png"
    plt.savefig(plot_filename)
    plt.close()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 2e-4, 5e-4
# lr_period, lr_decay, net = 4, 0.9, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# lr_period, lr_decay, net = 50, 0.1, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 50, 2e-4, 5e-4
# lr_period, lr_decay, net = 4, 0.9, get_net()
# # devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# # lr_period, lr_decay, net = 50, 0.1, get_net()
# train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
#       lr_decay)


Training on NVIDIA GeForce GTX 1660 Ti
Files already downloaded and verified
Files already downloaded and verified


In [100]:
import torch
import numpy as np
import random

def add_noise_to_labels(labels, noise_ratio, noise_type='symmetric', num_classes=10):
    if noise_type not in ['symmetric', 'asymmetric']:
        raise ValueError("noise_type should be 'symmetric' or 'asymmetric'")

    noisy_labels = labels.clone()
    n = len(labels)

    # 对称噪声
    if noise_type == 'symmetric':
        for i in range(n):
            if random.random() < noise_ratio:
                noisy_labels[i] = random.choice([l for l in range(num_classes) if l != labels[i]])

    # 非对称噪声
    elif noise_type == 'asymmetric':
        print("asy: ", n, " ", noise_ratio)
        # CIFAR-10 specific label mapping
        mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}
        for i in range(n):
            if random.random() < noise_ratio and labels[i].item() in mapping:
                # print("change")
                noisy_labels[i] = mapping[labels[i].item()]

    return noisy_labels
    
def verify_noise(original_labels, noisy_labels):
    assert len(original_labels) == len(noisy_labels), "Length of original and noisy labels must be equal"
    changed = (original_labels != noisy_labels).sum().item()
    total = len(original_labels)
    noise_ratio = changed / total
    return noise_ratio

# 假设 labels 是一个 PyTorch 张量，包含 CIFAR 数据集的标签

# 提取训练集的标签
original_labels = torch.tensor([full_train_ds[i][1] for i in range(len(full_train_ds))])

# 添加噪声
noise_ratio = 0.8  # 例如，添加 30% 的噪声
noisy_labels = add_noise_to_labels(original_labels, noise_ratio, noise_type='symmetric')

# 验证噪声添加
actual_noise_ratio = verify_noise(original_labels, noisy_labels)
print(f"Actual noise ratio: {actual_noise_ratio}")
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (symmetric):", noisy_labels[:10])
# 假设 original_labels 和 noisy_labels 分别是原始和带噪声的标签
# 定义非对称噪声的映射关系
asymmetric_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}

def calculate_asymmetric_noise_ratio(original_labels, noisy_labels, mapping):
    """计算实际的非对称噪声比例"""
    n = len(original_labels)
    actual_changes = 0
    potential_changes = 0

    for i in range(n):
        if original_labels[i].item() in mapping:
            potential_changes += 1
            if noisy_labels[i].item() == mapping[original_labels[i].item()]:
                actual_changes += 1

    if potential_changes == 0:
        return 0
    else:
        return actual_changes / potential_changes


noisy_labels_a = add_noise_to_labels(original_labels, noise_ratio, noise_type='asymmetric')
# 检查原始标签和噪声标签是否有差异
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (asymmetric):", noisy_labels_a[:10])


# 计算实际的非对称噪声比例
actual_asymmetric_noise_ratio = calculate_asymmetric_noise_ratio(original_labels, noisy_labels_a, asymmetric_mapping)
print(f"Actual asymmetric noise ratio: {actual_asymmetric_noise_ratio}")



Actual noise ratio: 0.79622
Some original labels: tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])
Corresponding noisy labels (symmetric): tensor([3, 9, 0, 3, 1, 3, 9, 9, 4, 1])
asy:  50000   0.8
Some original labels: tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])
Corresponding noisy labels (asymmetric): tensor([6, 1, 1, 4, 1, 1, 0, 7, 8, 3])
Actual asymmetric noise ratio: 0.7958


In [101]:
def run_train_harness(noise_level=0.0, noise_type='symmetric'):
    # 重新加载数据集以避免标签被多次修改
    full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
    original_labels = torch.tensor(full_train_ds.targets)

    # 应用噪声
    if noise_level > 0:
        noisy_labels = add_noise_to_labels(original_labels, noise_level, noise_type)
        full_train_ds.targets = noisy_labels.tolist()

    # 分割训练集和验证集
    num_train = len(full_train_ds)
    num_valid = int(num_train * valid_ratio)
    train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])

    # 创建 DataLoader
    train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    
    devices, num_epochs, lr, wd = d2l.try_all_gpus(), 40, 2e-4, 5e-4
    lr_period, lr_decay = 4, 0.9
    net = vgg(small_conv_arch)

    train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay)
    

# # 训练没有噪声的模型
print(f"####### Training model with 0 noise level: ########")
run_train_harness(0, 'symmetric')
print("\n\n\n")

# 用不同的噪声水平和类型训练模型
# noise_levels = [0.1, 0.3, 0.5, 0.8]
# noise_types = ['symmetric', 'asymmetric']

# for noise_type in noise_types:
#     for noise_level in noise_levels:
#         print(f"####### Training with {noise_type} noise level: {noise_level} ########")
#         run_train_harness(noise_level, noise_type)
#         print("\n\n\n")


####### Training model with 0 noise level: ########
Files already downloaded and verified
1/40 - 15s - loss: 2.3040 - accuracy: 0.0968 - val_accuracy: 0.0974 - 15s/epoch - 0ms/step
2/40 - 15s - loss: 2.3040 - accuracy: 0.0993 - val_accuracy: 0.0972 - 15s/epoch - 0ms/step
3/40 - 16s - loss: 2.3037 - accuracy: 0.1018 - val_accuracy: 0.0972 - 16s/epoch - 0ms/step
4/40 - 15s - loss: 2.1907 - accuracy: 0.1389 - val_accuracy: 0.2110 - 15s/epoch - 0ms/step
5/40 - 15s - loss: 1.9024 - accuracy: 0.2336 - val_accuracy: 0.2540 - 15s/epoch - 0ms/step
6/40 - 15s - loss: 1.6985 - accuracy: 0.3403 - val_accuracy: 0.3612 - 15s/epoch - 0ms/step
7/40 - 15s - loss: 1.5117 - accuracy: 0.4285 - val_accuracy: 0.4718 - 15s/epoch - 0ms/step
8/40 - 15s - loss: 1.3374 - accuracy: 0.5142 - val_accuracy: 0.5396 - 15s/epoch - 0ms/step
9/40 - 15s - loss: 1.1838 - accuracy: 0.5766 - val_accuracy: 0.5878 - 15s/epoch - 0ms/step
10/40 - 15s - loss: 1.0919 - accuracy: 0.6136 - val_accuracy: 0.6120 - 15s/epoch - 0ms/step

In [102]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.1
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.1 ########
Files already downloaded and verified
1/40 - 15s - loss: 2.3042 - accuracy: 0.0981 - val_accuracy: 0.1010 - 15s/epoch - 0ms/step
2/40 - 16s - loss: 2.3038 - accuracy: 0.0994 - val_accuracy: 0.0976 - 16s/epoch - 0ms/step
3/40 - 15s - loss: 2.3038 - accuracy: 0.0995 - val_accuracy: 0.1082 - 15s/epoch - 0ms/step
4/40 - 15s - loss: 2.3033 - accuracy: 0.0990 - val_accuracy: 0.1098 - 15s/epoch - 0ms/step
5/40 - 21s - loss: 2.1158 - accuracy: 0.1729 - val_accuracy: 0.1848 - 21s/epoch - 0ms/step
6/40 - 22s - loss: 1.9767 - accuracy: 0.2414 - val_accuracy: 0.2782 - 22s/epoch - 0ms/step
7/40 - 21s - loss: 1.8454 - accuracy: 0.3157 - val_accuracy: 0.3044 - 21s/epoch - 0ms/step
8/40 - 21s - loss: 1.7176 - accuracy: 0.3829 - val_accuracy: 0.4232 - 21s/epoch - 0ms/step
9/40 - 19s - loss: 1.5989 - accuracy: 0.4520 - val_accuracy: 0.4926 - 19s/epoch - 0ms/step
10/40 - 21s - loss: 1.5096 - accuracy: 0.4989 - val_accuracy: 0.5036 - 21s/epoch - 0m

In [103]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.3
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.3 ########
Files already downloaded and verified
1/40 - 14s - loss: 2.3041 - accuracy: 0.0984 - val_accuracy: 0.0990 - 14s/epoch - 0ms/step
2/40 - 15s - loss: 2.3038 - accuracy: 0.1022 - val_accuracy: 0.0978 - 15s/epoch - 0ms/step
3/40 - 14s - loss: 2.3035 - accuracy: 0.1041 - val_accuracy: 0.1016 - 14s/epoch - 0ms/step
4/40 - 14s - loss: 2.3037 - accuracy: 0.1007 - val_accuracy: 0.0986 - 14s/epoch - 0ms/step
5/40 - 15s - loss: 2.3034 - accuracy: 0.1016 - val_accuracy: 0.1016 - 15s/epoch - 0ms/step
6/40 - 15s - loss: 2.3035 - accuracy: 0.0990 - val_accuracy: 0.1040 - 15s/epoch - 0ms/step
7/40 - 15s - loss: 2.3034 - accuracy: 0.0994 - val_accuracy: 0.1020 - 15s/epoch - 0ms/step
8/40 - 15s - loss: 2.3035 - accuracy: 0.0979 - val_accuracy: 0.0968 - 15s/epoch - 0ms/step
9/40 - 14s - loss: 2.3032 - accuracy: 0.0998 - val_accuracy: 0.0986 - 14s/epoch - 0ms/step
10/40 - 15s - loss: 2.3029 - accuracy: 0.1006 - val_accuracy: 0.1016 - 15s/epoch - 0m

In [104]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.5
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.5 ########
Files already downloaded and verified
1/40 - 14s - loss: 2.3042 - accuracy: 0.1001 - val_accuracy: 0.0984 - 14s/epoch - 0ms/step
2/40 - 14s - loss: 2.3039 - accuracy: 0.1000 - val_accuracy: 0.1038 - 14s/epoch - 0ms/step
3/40 - 15s - loss: 2.3038 - accuracy: 0.0995 - val_accuracy: 0.0962 - 15s/epoch - 0ms/step
4/40 - 15s - loss: 2.3037 - accuracy: 0.0986 - val_accuracy: 0.1038 - 15s/epoch - 0ms/step
5/40 - 15s - loss: 2.3033 - accuracy: 0.1024 - val_accuracy: 0.1054 - 15s/epoch - 0ms/step
6/40 - 15s - loss: 2.3037 - accuracy: 0.0997 - val_accuracy: 0.0938 - 15s/epoch - 0ms/step
7/40 - 14s - loss: 2.3034 - accuracy: 0.1006 - val_accuracy: 0.0938 - 14s/epoch - 0ms/step
8/40 - 15s - loss: 2.3032 - accuracy: 0.0988 - val_accuracy: 0.1026 - 15s/epoch - 0ms/step
9/40 - 15s - loss: 2.3032 - accuracy: 0.1011 - val_accuracy: 0.0984 - 15s/epoch - 0ms/step
10/40 - 15s - loss: 2.3031 - accuracy: 0.0997 - val_accuracy: 0.0938 - 15s/epoch - 0m

In [105]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.8
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.8 ########
Files already downloaded and verified
1/40 - 52s - loss: 2.3040 - accuracy: 0.1001 - val_accuracy: 0.1022 - 52s/epoch - 0ms/step
2/40 - 52s - loss: 2.3040 - accuracy: 0.0994 - val_accuracy: 0.0996 - 52s/epoch - 0ms/step
3/40 - 52s - loss: 2.3037 - accuracy: 0.1004 - val_accuracy: 0.1028 - 52s/epoch - 0ms/step
4/40 - 52s - loss: 2.3036 - accuracy: 0.1002 - val_accuracy: 0.1012 - 52s/epoch - 0ms/step
5/40 - 52s - loss: 2.3034 - accuracy: 0.1016 - val_accuracy: 0.0968 - 52s/epoch - 0ms/step
6/40 - 52s - loss: 2.3034 - accuracy: 0.0999 - val_accuracy: 0.1012 - 52s/epoch - 0ms/step
7/40 - 52s - loss: 2.3033 - accuracy: 0.0975 - val_accuracy: 0.0996 - 52s/epoch - 0ms/step
8/40 - 52s - loss: 2.3033 - accuracy: 0.1008 - val_accuracy: 0.1000 - 52s/epoch - 0ms/step
9/40 - 52s - loss: 2.3031 - accuracy: 0.0999 - val_accuracy: 0.1000 - 52s/epoch - 0ms/step
10/40 - 52s - loss: 2.3031 - accuracy: 0.0987 - val_accuracy: 0.0968 - 52s/epoch - 0m

In [106]:
noise_types = ['asymmetric']
for noise_type in noise_types:
    noise_level = 0.8
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with asymmetric noise level: 0.8 ########
Files already downloaded and verified
asy:  50000   0.8
1/40 - 37s - loss: 2.0909 - accuracy: 0.1774 - val_accuracy: 0.1808 - 37s/epoch - 0ms/step
2/40 - 37s - loss: 2.0818 - accuracy: 0.1804 - val_accuracy: 0.1784 - 37s/epoch - 0ms/step


KeyboardInterrupt: 